In [6]:
#load libraries
import pandas as pd
import os
import numpy as np
from datetime import datetime
from ultralytics import YOLO
from tqdm import tqdm
import cv2

In [4]:
#import images and models
file_path = 'home/lero/idrive/cmac/DDMAP/Stability studies'

folders = ['40_C_75_RH',
           '40_C_75_RH',
           '30_C_30_RH'
          ]

wells_detection_model_path= '/home/lero/idrive/cmac/DDMAP/Image_analysis/Code/wells_model/Results/fold_3/weights/best.pt'
crystal_detection_model_path = '/home/lero/idrive/cmac/DDMAP/Image_analysis/Code/cryst_amorphous_model/Results/fold_1/weights/best.pt'

wells_detection_model = YOLO(wells_detection_model_path)
crystal_detection_model = YOLO(crystal_detection_model_path)

ignored_classes = {'Label'}

In [5]:
#create functions

# Function to convert filename timestamp to datetime object
def convert_timestamp(filename):
    # ignore extension (last 4 chars)
    return datetime.strptime(filename[:-4], "%Y%m%d_%H%M%S")

def calculate_mean_intensity (image, x1, x2, y1, y2):
    roi = image[y1:y2, x1:x2] #region of interest
    gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY) #greyscale
    return np.mean(gray_roi.astype(float)) #calculate mean pixel intensity

# Color coding for object classes
class_colors = {
    "Dust": (255, 0, 0),  # Red
    "empty_well": (0, 255, 0),  # Green
    "Crystal": (255, 255, 0)  # Cyan
}


In [ ]:
for folder in folders:
    folder_path = os.path.join(file_path, folder)
    #Image pathway
    for api in os.listdir(folder_path):
        image_folder = os.path.join(folder_path, api)
        if not os.path.isdir(image_folder):
            continue
        print(f'processing {api} in folder {folder}')
        #output folder
        output_folder = os.path.join(image_folder, 'new_results')
        os.makedir(output_folder, exist_ok=True)

        #get sorted images
        images = sorted(img for img in os.listdir(image_folder) if img.endswith(('.png'))
        if not images:
            print(f'no images found in {image_folder}, skipping')
            continue

        #get the first image timestamp
        assert len(images)
        first_image_timestamp = convert_timestamp(images[0])
        first_image_timestamp_str = first_image_timestamp.strftime("%d-%m-%Y %H:%M")

        # Dictionary to track object states
        object_last_state = {}
        object_detections = {}
        object_positions = []

        wells = list(range(96))
        well_data = {wi: [], for wi in wells}
        stability_results = {wi:{'Timestamp':None, 'Class': None} for wi in wells}
        WINDOW = 40

        #read and process images
        for idx, image_name in enumerate(tqdm(images, unit='images')):
            image_path = os.path.join(image_folder, image_name)
            img = cv2.imread(image_path) #image of well plate
            img_height, img_width = img.shape[:2]

            results = wells_detection_model(img)
            located_wells = []

            for result in results:
                for box in result.boxes:
                    cls_id = int(box.cls)
                    cls_name = result.names[cls_id]
                    conf = round(float(box.conf), 2)

                    if cls_name in ignored_classes:
                        continue

                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
                    located_wells.append((x1, y1, x2, y2, center_x, center_y, cls_name, conf)) 
            
            #normalise grid based on well [0] and map all wells into grid positions
            if located_wells:
                x = [o[4] for o in located_wells if o[-1] > 0.75]
                y = [o[5] for o in located_wells if o[-1] > 0.75]
                left = min(x)
                right = max(x)
                top = min(y)
                bottom = max(y)
                width = right - left
                height = bottom - top
                reference_well = min(located_wells, key=lambda o: (o[5], o[4])) #since origin of image is (0,0)
                ref_x, ref_y = reference_well[4], reference_well[5]

                normalised_wells = []
                for obj in located_wells:
                    x1, y1, x2, y2, center_x, center_y, cls_name, conf = obj
                    midp_x = (x1+x2)/2
                    midp_y = (y1+y2)/2
                    x_coord = int((midp_x - left)*12/width/1.03)
                    y_coord = int((midp_y - top)*8/height)
                    x_coord = min(11, max(0, x_coord)) #set min and max rows from 0,11
                    y_coord = min(7, max(0 y_coord)) #set min and max columns from 0,7
                    normalised_wells.append(x1, y1, x2, y2, center_x, center_y, cls_name, conf, x_coord, y_coord)
                
                #keep only the best detections per grid 
                best_detections = {}
                for obj in normalised_wells:
                    x1, y1, x2, y2, center_x, center_y, cls_name, conf, x_coord, y_coord = obj
                    well_key = (x_coord, y_coord)
                    if well_key not in best_detections or conf > well_key[7]:
                        best_detections[well_key] = obj
                normalised_wells = list(best_detections.values())
            else:
                normalised_wells = []
                    
                    
                
                                  
                    
                
                
            
                    
                                         
            





        